In [0]:
data = sc.textFile("/FileStore/tables/soc_LiveJournal1Adj.txt")

In [0]:
data.take(10)

Out[4]: ['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878',
 '3\t0,12,41,55,1532,12636,13185,27552,38737',
 '4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822',
 '5\t0,1,20,2022,22939,23527,30257,32503,35633,41457,43262,44846,49574,31140,32828',
 '6\t0,21,98

In [0]:
def createUserFriendStructure(x):
    userId = x.split()[0]
    try:
        friendId = list(x.split()[1].split(','))
    except:
        friendId = []
    return (userId, friendId)

In [0]:
#(user, [list of friends])
userFriendsStructure = data.map(lambda x: createUserFriendStructure(x))
userFriendsStructure.count()

Out[29]: 49995

In [0]:
#create ((Id1, Id2), 0) if already friends ans ((Id1, Id2), 1) if not 
import itertools
def createFriendStructure(row):
    print('here')
    userId = row[0]
    friends = row[1]
 
    connections = []
    for friendId in friends:
        key = (userId, friendId)
        if int(userId) > int(friendId):
            key = (friendId, userId)
        connections.append((key , 0))
    
    for friendIds in itertools.combinations(friends, 2):
        key = (friendIds[0], friendIds[1])
        if int(friendIds[0]) > int(friendIds[1]):
            key = (friendIds[1], friendIds[0])
        connections.append((key, 1))
    return connections


In [0]:
mutualFriendStructure = userFriendsStructure.flatMap(lambda x: createFriendStructure(x))
mutualFriendStructure.count()

Out[30]: 12116139

In [0]:
notFriendsWithMutualFriends = mutualFriendStructure.groupByKey().filter(lambda x: 0 not in x[1]).map(lambda x: (x[0], sum(x[1])))
notFriendsWithMutualFriends.count()

Out[31]: 6027098

In [0]:
def friendsWithFriendIDAndCount(row):
    users = row[0]
    count = row[1]
    recommendation1 = (users[0], (users[1], count))
    recommendation2 = (users[1], (users[0], count))
    return [recommendation1, recommendation2]

In [0]:
userWiseMutualFriendList = notFriendsWithMutualFriends.flatMap(lambda x: friendsWithFriendIDAndCount(x))
userWiseMutualFriendList.take(5)

Out[32]: [('32210', ('34389', 1)),
 ('34389', ('32210', 1)),
 ('13845', ('32717', 1)),
 ('32717', ('13845', 1)),
 ('23305', ('45195', 1))]

In [0]:
def getTop10MutualFriends(rows):
    try:
        rows = list(rows)
        rows.sort(key = lambda x: (-x[1], x[0]))
        return list(map(lambda x: x[0], rows))[:10]
    except:
        print("error")

In [0]:
#groupbykey and sort, select first 10
recommendations = userWiseMutualFriendList.groupByKey().map(lambda x: (x[0], getTop10MutualFriends(x[1])))

In [0]:
recommendations.take(10)

Out[14]: [('35066',
  ['35112',
   '35114',
   '35057',
   '35083',
   '35097',
   '35098',
   '35101',
   '35105',
   '35113',
   '35122']),
 ('8347',
  ['43242',
   '43438',
   '10301',
   '12598',
   '134',
   '43171',
   '43218',
   '43291',
   '43575',
   '46792']),
 ('23911',
  ['13016',
   '20145',
   '39476',
   '40615',
   '7690',
   '12060',
   '13278',
   '13533',
   '14582',
   '17594']),
 ('11570',
  ['4289',
   '4295',
   '4264',
   '19077',
   '22145',
   '22163',
   '4074',
   '4202',
   '4246',
   '22098']),
 ('14154',
  ['14013',
   '14150',
   '13948',
   '14132',
   '13917',
   '13981',
   '14073',
   '14123',
   '14131',
   '14164']),
 ('45626',
  ['45598',
   '45602',
   '6882',
   '1111',
   '15989',
   '21307',
   '23476',
   '2389',
   '24249',
   '33955']),
 ('22546',
  ['10494',
   '22508',
   '22532',
   '22540',
   '22541',
   '11214',
   '13267',
   '13618',
   '13654',
   '13665']),
 ('35811',
  ['9770',
   '14244',
   '35612',
   '35731',
   '9798',
   '

In [0]:
recommendations.collect()

Out[22]: [('4760',
  ['1918',
   '4717',
   '4718',
   '4719',
   '4720',
   '4721',
   '4722',
   '4723',
   '4724',
   '4725']),
 ('20487',
  ['39135',
   '12068',
   '12187',
   '13280',
   '14280',
   '14371',
   '14569',
   '14691',
   '14991',
   '15175']),
 ('34146',
  ['14246',
   '34204',
   '34439',
   '6783',
   '34201',
   '34207',
   '34266',
   '34297',
   '11377',
   '13872']),
 ('21171',
  ['19554',
   '7651',
   '16539',
   '20899',
   '4736',
   '10500',
   '20852',
   '546',
   '10469',
   '14274']),
 ('11380',
  ['11395',
   '11377',
   '11401',
   '11410',
   '11420',
   '11425',
   '11381',
   '11384',
   '11385',
   '11388']),
 ('34772',
  ['14178',
   '8726',
   '14040',
   '14131',
   '14132',
   '23611',
   '23689',
   '34319',
   '34347',
   '34566']),
 ('41059',
  ['12107',
   '29869',
   '35851',
   '11591',
   '12632',
   '12636',
   '12640',
   '12641',
   '12655',
   '12706']),
 ('12156',
  ['12125',
   '12131',
   '12135',
   '12136',
   '12137',
   '12

In [0]:
formatted_recommendations = recommendations.map(lambda x: f"{x[0]}\t{', '.join(x[1])}")

In [0]:
formatted_recommendations.take(10)

Out[34]: ['20585\t30195, 13223, 13681, 14264, 15310, 16104, 1680, 18109, 18110, 22402',
 '8319\t8301, 8362, 10232, 37973, 8284, 8286, 8291, 8295, 501, 8336',
 '35551\t32203, 35738, 35750, 38794, 9778, 12653, 1417, 15289, 18301, 18437',
 '33268\t11779, 10686, 11205, 14447, 16483, 17576, 28820, 32741, 33260, 33281',
 '46268\t46278, 46295, 39631, 39635, 39670, 13803, 13889, 15911, 1889, 20214',
 '43019\t43063, 22506, 30274, 11863, 13280, 14324, 15761, 17468, 17478, 20505',
 '19251\t19234, 19236, 19238, 19239, 19247, 19248, 19252, 19221, 19222, 19223',
 '34976\t34921, 34916, 34917, 34923, 34968, 34972, 34978, 12466, 15817, 17203',
 '47856\t26090, 26066, 26057, 26073, 47857, 26089, 26054, 16389, 16412, 26058',
 '19033\t20599, 19107, 146, 19011, 19032, 24813, 2773, 6187, 100, 107']

In [0]:
#to store in a file
finalRecommendations = recommendations.sortByKey()

In [0]:
finalRecommendations.saveAsTextFile("/FileStore/tables/Recommendations.txt")